<a href="https://colab.research.google.com/github/phonhay103/anything/blob/main/MobileNet_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import tensorflow as tf

In [51]:
def make_divisible(v, divisor, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v

def correct_pad(inputs, kernel_size):
    input_size = inputs.shape[1:3]
    if input_size[0] is None:
        adjust = (1, 1)
    else:
        adjust = (1 - input_size[0] % 2, 1 - input_size[1] % 2)
    correct = (kernel_size // 2, kernel_size // 2)
    return ((correct[0] - adjust[0], correct[0]),
            (correct[1] - adjust[1], correct[1]))

In [56]:
def inverted_res_block(inputs, expansion, stride, alpha, filters, block_id):
    """Inverted ResNet block."""
    in_channels = inputs.shape[-1]
    pointwise_conv_filters = int(filters * alpha)
    pointwise_filters = make_divisible(pointwise_conv_filters, 8)
    x = inputs
    prefix = 'block_{}_'.format(block_id)

    # Expand
    if block_id:
        x = tf.keras.layers.Conv2D(
            expansion * in_channels,
            kernel_size=1,
            padding='same',
            use_bias=False,
            name=prefix + 'expand')(x)
        x = tf.keras.layers.BatchNormalization(
            epsilon=1e-3,
            momentum=0.999,
            name=prefix + 'expand_BN')(x)
        x = tf.keras.layers.ReLU(6., name=prefix + 'expand_relu')(x)
    else:
        prefix = 'expanded_conv_'

    # Depthwise
    if stride == 2:
        x = tf.keras.layers.ZeroPadding2D(
            padding=correct_pad(x, 3),
            name=prefix + 'pad')(x)

    x = tf.keras.layers.DepthwiseConv2D(
        kernel_size=3,
        strides=stride,
        use_bias=False,
        padding='same' if stride == 1 else 'valid',
        name=prefix + 'depthwise')(x)
    x = tf.keras.layers.BatchNormalization(
        epsilon=1e-3,
        momentum=0.999,
        name=prefix + 'depthwise_BN')(x)
    x = tf.keras.layers.ReLU(6., name=prefix + 'depthwise_relu')(x)

    # Project
    x = tf.keras.layers.Conv2D(
        pointwise_filters,
        kernel_size=1,
        padding='same',
        use_bias=False,
        name=prefix + 'project')(x)
    x = tf.keras.layers.BatchNormalization(
        epsilon=1e-3,
        momentum=0.999,
        name=prefix + 'project_BN')(x)

    if in_channels == pointwise_filters and stride == 1:
        return tf.keras.layers.Add(name=prefix + 'add')([inputs, x])
    return x

In [57]:
def MobileNetV2(input_shape=None, alpha=1.0, classes=1000, classifier_activation='softmax'):
    # Input Shape
    if input_shape is None:
        input_shape = (224, 224, 3)
    else:
        rows = input_shape[0]
        cols = input_shape[1]
        if rows == cols and rows in [128, 160, 192, 224]:
            input_shape = (rows, rows, 3)
        else:
            input_shape = (224, 224, 3)
    
    # Feature Extraction
    img_input = tf.keras.layers.Input(shape=input_shape, name='input')
    x = tf.keras.layers.Conv2D(make_divisible(32 * alpha, 8), kernel_size=3, strides=(2, 2), padding='same', use_bias=False, name='First_conv')(img_input)
    x = tf.keras.layers.BatchNormalization(epsilon=1e-3, momentum=0.999, name='First_conv_bn')(x)
    x = tf.keras.layers.ReLU(6., name='First_conv_relu')(x)
    x = inverted_res_block(x, filters=16, alpha=alpha, stride=1, expansion=1, block_id=0)
    x = inverted_res_block(x, filters=24, alpha=alpha, stride=2, expansion=6, block_id=1)
    x = inverted_res_block(x, filters=24, alpha=alpha, stride=1, expansion=6, block_id=2)
    x = inverted_res_block(x, filters=32, alpha=alpha, stride=2, expansion=6, block_id=3)
    x = inverted_res_block(x, filters=32, alpha=alpha, stride=1, expansion=6, block_id=4)
    x = inverted_res_block(x, filters=32, alpha=alpha, stride=1, expansion=6, block_id=5)
    x = inverted_res_block(x, filters=64, alpha=alpha, stride=2, expansion=6, block_id=6)
    x = inverted_res_block(x, filters=64, alpha=alpha, stride=1, expansion=6, block_id=7)
    x = inverted_res_block(x, filters=64, alpha=alpha, stride=1, expansion=6, block_id=8)
    x = inverted_res_block(x, filters=64, alpha=alpha, stride=1, expansion=6, block_id=9)
    x = inverted_res_block(x, filters=96, alpha=alpha, stride=1, expansion=6, block_id=10)
    x = inverted_res_block(x, filters=96, alpha=alpha, stride=1, expansion=6, block_id=11)
    x = inverted_res_block(x, filters=96, alpha=alpha, stride=1, expansion=6, block_id=12)
    x = inverted_res_block(x, filters=160, alpha=alpha, stride=2, expansion=6, block_id=13)
    x = inverted_res_block(x, filters=160, alpha=alpha, stride=1, expansion=6, block_id=14)
    x = inverted_res_block(x, filters=160, alpha=alpha, stride=1, expansion=6, block_id=15)
    x = inverted_res_block(x, filters=320, alpha=alpha, stride=1, expansion=6, block_id=16)

    if alpha > 1.0:
        last_block_filters = make_divisible(1280 * alpha, 8)
    else:
        last_block_filters = 1280
    x = tf.keras.layers.Conv2D(last_block_filters, kernel_size=1, use_bias=False, name='Last_conv')(x)
    x = tf.keras.layers.BatchNormalization(epsilon=1e-3, momentum=0.999, name='Last_conv_bn')(x)
    x = tf.keras.layers.ReLU(6., name='Last_conv_relu')(x)

    # Classification
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(classes, activation=classifier_activation, name='predictions')(x)
    
    # Model
    return tf.keras.Model(inputs=img_input, outputs=x)

In [58]:
model = MobileNetV2()

In [59]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
First_conv (Conv2D)             (None, 112, 112, 32) 864         input[0][0]                      
__________________________________________________________________________________________________
First_conv_bn (BatchNormalizati (None, 112, 112, 32) 128         First_conv[0][0]                 
__________________________________________________________________________________________________
First_conv_relu (ReLU)          (None, 112, 112, 32) 0           First_conv_bn[0][0]              
____________________________________________________________________________________________

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['acc'])